In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling as pp
from scipy import stats
import scipy
from sklearn.linear_model import LogisticRegression, LinearRegression
import itertools
from pandas.plotting import scatter_matrix
from sklearn.metrics import accuracy_score
from random import sample 
#from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

### Loading the data:

In [6]:
df = pd.read_csv('./Telco_Churn_data.csv', sep=',')
df.dtypes


customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

### Checking data profile:

In [3]:
profile = pp.ProfileReport(df, title='Pandas Profiling Report', html={'style':{'full_width':True}})
profile.to_file(output_file="your_report.html")

### Preparing data for the model:

Dealing with 'Yes' 'No' Values

In [7]:
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})
df['Dependents'] = df['Dependents'].map({'Yes': 1, 'No': 0})
df['PaperlessBilling'] = df['PaperlessBilling'].map({'Yes': 1, 'No': 0})
df['Partner'] = df['Partner'].map({'Yes': 1, 'No': 0})
df['PhoneService'] = df['PhoneService'].map({'Yes': 1, 'No': 0})

Cleaning the data

In [8]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'].replace(' ', 0))
df = df.drop(axis=1, columns=['customerID'])

Dealing with Categorical Data

In [9]:
df = pd.get_dummies(df)

### Building the model:

In [10]:
X = df.loc[:, df.columns != 'Churn']
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

### LogisticRegression:

In [11]:
def logistic(x):
    return 1/(1 + np.exp(-x))

npts = 100
beta0 = 0.0
beta1 = 0.5

model = LogisticRegression(C=1000)
model.fit(X, y)
beta0hat = model.intercept_[0]
beta1hat = model.coef_[0][0]
print("beta0 =    {0:8.3f} beta1 =    {1:8.3f}".format(beta0, beta1))
print("beta0hat = {0:8.3f} beta1hat = {1:8.3f}".format(beta0hat, beta1hat))

y_pred = model.predict(X_test)
predictions = [value for value in y_pred]
accuracy = accuracy_score(y_test, predictions)

print("Accuracy: %.2f%%" % (accuracy * 100.0))
dataColumns=[] 
for col in X.columns: 
    dataColumns.append(col)

print(classification_report(y_test, predictions,target_names=['0','1']))
unique_label = np.unique(y_test)
print(pd.DataFrame(confusion_matrix(y_test, predictions, labels=unique_label), 
                   index=['true:{:}'.format(x) for x in unique_label], 
                   columns=['pred:{:}'.format(x) for x in unique_label]))

beta0 =       0.000 beta1 =       0.500
beta0hat =   -0.178 beta1hat =    0.218
Accuracy: 80.70%
              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1021
           1       0.68      0.55      0.61       388

    accuracy                           0.81      1409
   macro avg       0.76      0.73      0.74      1409
weighted avg       0.80      0.81      0.80      1409

        pred:0  pred:1
true:0     922      99
true:1     173     215
